### 🔹 Step 1: Install Required Libraries

**pip install lightfm pandas numpy**


In [ ]:
pip install lightfm pandas numpy


### 🔹 Step 2: Load Your Dataset (events.csv)

In [1]:

import pandas as pd

# Load dataset
events = pd.read_csv("data/events.csv")

# Display first few rows
print(events.head())


       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN


### 🔹 Step 3: Convert Data for LightFM

In [2]:
from lightfm import LightFM
from lightfm.data import Dataset

# Initialize LightFM dataset
dataset = Dataset()

# Fit dataset with unique users and items
dataset.fit(users=events["visitorid"].unique(), items=events["itemid"].unique())

# Build interaction matrix (assuming each event counts as an interaction)
(interactions, weights) = dataset.build_interactions(
    (row["visitorid"], row["itemid"]) for _, row in events.iterrows()
)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### 🔹 Step 4: Train a Simple Recommendation Model

In [3]:
# Initialize model
model = LightFM(loss="warp")  # WARP optimizes ranking

# Train model for 10 epochs
model.fit(interactions, epochs=10, num_threads=2)


### 🔹 Step 5: Generate Recommendations

In [4]:
import numpy as np

def recommend_items(model, dataset, user_id, n=5):
    """Recommend top N items for a given user."""
    n_users, n_items = dataset.interactions_shape()
    
    # Predict scores for all items
    scores = model.predict(user_id, np.arange(n_items))
    
    # Get top N items
    top_items = np.argsort(-scores)[:n]
    
    # Map back to item IDs
    item_mapping = list(dataset.mapping()[2].keys())
    recommended_items = [item_mapping[i] for i in top_items]
    
    return recommended_items

# Example: Get recommendations for user ID 257597
print("Recommended items:", recommend_items(model, dataset, user_id=0, n=5))


Recommended items: [247909, 153778, 65273, 315545, 253615]
